# Loan Application Orchestration Pipeline Demo (Agent Orchestration)

This notebook demonstrates the full loan application decision process **using the agent orchestration pipeline**. The orchestration function calls all tools and combines the results into a final summary.

In [2]:
from dotenv import load_dotenv
load_dotenv(".env", override=True)

# Import required modules and orchestration pipeline
from data_model import LoanApplicationJourney
from utils.config_loader import ConfigLoader
from orchestrator_pipeline import orchestrate_application
from tools.synthesize_summary import synthesize_summary
from agents import RunContextWrapper

# Import agent objects
from agents.fraud_agent import fraud_agent
from agents.sla_agent import sla_agent
from agents.recommendation_agent import recommendation_agent
from agents.orchestrator_agent import orchestrator_agent
from agents.report_agent import report_agent
from agents.interest_rate_agent import interest_rate_agent

#### Test example

In [3]:
# Create a test LoanApplicationJourney instance
test_app = LoanApplicationJourney(
    application_id="APP-LOCAL-001",
    submitted_time="2025-06-01T09:00:00",
    reviewed_time="2025-06-01T09:30:00",
    approved_time="2025-06-01T10:00:00",
    rejected_time=None,
    processing_steps={"KYC": 72, "CreditCheck": 50, "FinalApproval": 35},
    flagged_for_fraud=False,
    monthly_income=50000,
    monthly_costs=1000,
    requested_amount=25000,
    monthly_debt=400
)

In [ ]:
# # Run the agent orchestration pipeline
# config = ConfigLoader()
# report = orchestrate_application(test_app, config=config)
# summary = synthesize_summary(RunContextWrapper(report))

In [ ]:
# summary

## Integrate and demonstrate the agents

In [4]:
# Example: Use interest_rate_agent to check for interest rate (sync call for demo)
interest_rate_result = interest_rate_agent.tools[0](RunContextWrapper(test_app))
print('Interest Rate Agent Result:', interest_rate_result)


Interest Rate Agent Result: {'rate': 4.250999927520752, 'date': '2025-07-01', 'source': 'Yahoo Finance (^TNX)'}


/Users/dariaz/Desktop/Oxford AI Summit - Full code summit assignment/Loan-application-agent/agents/interest_rate_agent.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  latest_rate = hist['Close'][-1]


In [5]:
# Example: Use fraud_agent to check for fraud indicators (sync call for demo)
fraud_result = fraud_agent.tools[0](RunContextWrapper(test_app))
print('Fraud Agent Result:', fraud_result)

Fraud Agent Result: {'flagged': False, 'label': 'No fraud indicators detected', 'risk_score': 0.0, 'triggered_rules': [], 'explanation': 'No indicators present'}


In [6]:
# Example: Use sla_agent to check SLA compliance (sync call for demo)
sla_result = sla_agent.tools[0](RunContextWrapper(test_app))
print('SLA Agent Result:', sla_result)

SLA Agent Result: {'violated': True, 'label': 'SLA violations in: KYC', 'violated_steps': ['KYC'], 'explanation': 'Steps exceeded configured SLA limits.'}


In [7]:
# Example: Use recommendation_agent to get a recommendation (sync call for demo)
rec_result = recommendation_agent.run('Provide recommendation', context=test_app)
print('Recommendation Agent Result:', getattr(rec_result, 'final_output', rec_result))

Recommendation Agent Result: <coroutine object Agent.run at 0x11d7e5230>


In [11]:
# Example: Use orchestrator_agent to run the full orchestration (sync call for demo)
orc_result = orchestrator_agent.tools[0](RunContextWrapper(test_app))
print('Orchestrator Agent Result:', orc_result)

Orchestrator Agent Result: {'violated': True, 'label': 'SLA violations in: KYC', 'violated_steps': ['KYC'], 'explanation': 'Steps exceeded configured SLA limits.'}


In [ ]:
# Example: Use report_agent to synthesize the summary (sync call for demo)
rep_result = report_agent.tools[0](RunContextWrapper(report))
print('Report Agent Result:', rep_result)

# TO DO:

## 1.Trend Analysis

### ToDo: Ask agents to analyze and take decisions based on the metrics below: 
####- interest rate cutoff : search the web to find what a reasonable interest rate is for a loan of this type -> 
####- fraud detection: based on the kyc and other metrics 
#- SLA compliance

### Goal: give AI more freedom to decide BY REMOVING THE CONFIG 

## 2.MCP Tools 
### Send email with report 